In [1]:
from pint import UnitRegistry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
G0 = 270
k1 = 0.02
k2 = 0.02
k3 = 1.5e-05

In [43]:
params = G0, k1, k2, k3

In [ ]:
def update_func(params):
    G0, k1, k2, k3 = params